In [1]:
import pandas as pd
import numpy as np
import requests
import os
import zipfile
import sys 
import json
import xlsxwriter
import psycopg2
from sqlalchemy import create_engine
import openpyxl

In [3]:
# detroit_zip = 'detroit_911_calls.zip'
# extracted_dir = 'detroit_911_calls'
# print( dir(zipfile))
# with zipfile.ZipFile(detroit_zip, 'r') as zipdir:
#         zipdir.extractall(extracted_dir)

['BZIP2_VERSION', 'BadZipFile', 'BadZipfile', 'CompleteDirs', 'DEFAULT_VERSION', 'LZMACompressor', 'LZMADecompressor', 'LZMA_VERSION', 'LargeZipFile', 'MAX_EXTRACT_VERSION', 'Path', 'PyZipFile', 'ZIP64_LIMIT', 'ZIP64_VERSION', 'ZIP_BZIP2', 'ZIP_DEFLATED', 'ZIP_FILECOUNT_LIMIT', 'ZIP_LZMA', 'ZIP_MAX_COMMENT', 'ZIP_STORED', 'ZipExtFile', 'ZipFile', 'ZipInfo', '_CD64_CREATE_VERSION', '_CD64_DIRECTORY_RECSIZE', '_CD64_DIRECTORY_SIZE', '_CD64_DISK_NUMBER', '_CD64_DISK_NUMBER_START', '_CD64_EXTRACT_VERSION', '_CD64_NUMBER_ENTRIES_THIS_DISK', '_CD64_NUMBER_ENTRIES_TOTAL', '_CD64_OFFSET_START_CENTDIR', '_CD64_SIGNATURE', '_CD_COMMENT_LENGTH', '_CD_COMPRESSED_SIZE', '_CD_COMPRESS_TYPE', '_CD_CRC', '_CD_CREATE_SYSTEM', '_CD_CREATE_VERSION', '_CD_DATE', '_CD_DISK_NUMBER_START', '_CD_EXTERNAL_FILE_ATTRIBUTES', '_CD_EXTRACT_SYSTEM', '_CD_EXTRACT_VERSION', '_CD_EXTRA_FIELD_LENGTH', '_CD_FILENAME_LENGTH', '_CD_FLAG_BITS', '_CD_INTERNAL_FILE_ATTRIBUTES', '_CD_LOCAL_HEADER_OFFSET', '_CD_SIGNATURE', '_C

In [2]:
detroit_calls = r'detroit_911_calls/dpd_911_calls_for_service.csv'

In [3]:
calls1 = pd.read_csv(detroit_calls, delimiter=",")


In [5]:
calls1_columns = calls1.columns
print( calls1_columns )
print( calls1.info() )

Index(['callno', 'agency', 'priority', 'incident_address', 'callcode',
       'calldescription', 'category', 'calldate', 'calltime', 'disposition',
       'precinctsca', 'respondingunit', 'officerinitiated', 'intaketime',
       'dispatchtime', 'traveltime', 'totresponsetime', 'timeonscene',
       'totaltime', 'location'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466260 entries, 0 to 466259
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   callno            466260 non-null  object 
 1   agency            466260 non-null  object 
 2   priority          466260 non-null  int64  
 3   incident_address  463368 non-null  object 
 4   callcode          466260 non-null  object 
 5   calldescription   466260 non-null  object 
 6   category          466260 non-null  object 
 7   calldate          466260 non-null  object 
 8   calltime          466260 non-null  object 
 9   disposit

In [9]:
for column in calls1_columns:
        print(f' {column} = {len(calls1[column].unique())} ')

 callno = 466101 
 agency = 3 
 priority = 23 
 incident_address = 56304 
 callcode = 293 
 calldescription = 327 
 category = 35 
 calldate = 180 
 calltime = 84917 
 disposition = 48 
 precinctsca = 133 
 respondingunit = 1750 
 officerinitiated = 2 
 intaketime = 289 
 dispatchtime = 4803 
 traveltime = 1443 
 totresponsetime = 5047 
 timeonscene = 4208 
 totaltime = 5973 
 location = 112344 


In [19]:
calls1["agency"] = calls1["agency"].astype("category")
calls1["callcode"] = calls1["callcode"].astype("category")
calls1["category"] = calls1["category"].astype("category")
calls1["disposition"] = calls1["disposition"].astype("category")
# calls1["officerinitiated"] = calls1["officerinitiated"].astype("category")

In [20]:
print( calls1.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466260 entries, 0 to 466259
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   callno            466260 non-null  object  
 1   agency            466260 non-null  category
 2   priority          466260 non-null  int64   
 3   incident_address  463368 non-null  object  
 4   callcode          466260 non-null  category
 5   calldescription   466260 non-null  object  
 6   category          466260 non-null  category
 7   calldate          466260 non-null  object  
 8   calltime          466260 non-null  object  
 9   disposition       265695 non-null  category
 10  precinctsca       390750 non-null  float64 
 11  respondingunit    227425 non-null  object  
 12  officerinitiated  466260 non-null  category
 13  intaketime        466260 non-null  float64 
 14  dispatchtime      466260 non-null  float64 
 15  traveltime        466260 non-null  float64 
 16  to

In [24]:
calls1["priority"] = calls1["priority"].astype("int16")
calls1["precinctsca"] = calls1["precinctsca"].astype("float32")
calls1["intaketime"] = calls1["intaketime"].astype("float32")
calls1["dispatchtime"] = calls1["dispatchtime"].astype("float32")
calls1["traveltime"] = calls1["traveltime"].astype("float32")
calls1["totresponsetime"] = calls1["totresponsetime"].astype("float32")
calls1["timeonscene"] = calls1["timeonscene"].astype("float32")
calls1["totaltime"] = calls1["totaltime"].astype("float32")
print( calls1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466260 entries, 0 to 466259
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   callno            466260 non-null  object  
 1   agency            466260 non-null  category
 2   priority          466260 non-null  int16   
 3   incident_address  463368 non-null  object  
 4   callcode          466260 non-null  category
 5   calldescription   466260 non-null  object  
 6   category          466260 non-null  category
 7   calldate          466260 non-null  object  
 8   calltime          466260 non-null  object  
 9   disposition       265695 non-null  category
 10  precinctsca       390750 non-null  float32 
 11  respondingunit    227425 non-null  object  
 12  officerinitiated  466260 non-null  category
 13  intaketime        466260 non-null  float32 
 14  dispatchtime      466260 non-null  float32 
 15  traveltime        466260 non-null  float32 
 16  to

In [25]:
calls1["precinctsca"] = calls1["precinctsca"].astype("float16")
calls1["intaketime"] = calls1["intaketime"].astype("float16")
calls1["dispatchtime"] = calls1["dispatchtime"].astype("float16")
calls1["traveltime"] = calls1["traveltime"].astype("float16")
calls1["totresponsetime"] = calls1["totresponsetime"].astype("float16")
calls1["timeonscene"] = calls1["timeonscene"].astype("float16")
calls1["totaltime"] = calls1["totaltime"].astype("float16")
print( calls1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466260 entries, 0 to 466259
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   callno            466260 non-null  object  
 1   agency            466260 non-null  category
 2   priority          466260 non-null  int16   
 3   incident_address  463368 non-null  object  
 4   callcode          466260 non-null  category
 5   calldescription   466260 non-null  object  
 6   category          466260 non-null  category
 7   calldate          466260 non-null  object  
 8   calltime          466260 non-null  object  
 9   disposition       265695 non-null  category
 10  precinctsca       390750 non-null  float16 
 11  respondingunit    227425 non-null  object  
 12  officerinitiated  466260 non-null  category
 13  intaketime        466260 non-null  float16 
 14  dispatchtime      466260 non-null  float16 
 15  traveltime        466260 non-null  float16 
 16  to